In [5]:
import os
import pandas as pd
import ast
# Specify the directory containing the Excel files
directory = 'C:/Users/boran/Desktop/Network/FTS Project/Son/Ülke Verileri/Ülke Verileri/'



In [6]:
# Tüm ülkelerin verilerini birleştiriyor. 
# Create an empty DataFrame to store aggregated data
aggregated_df = pd.DataFrame()

# Function to extract the appropriate word from the filename
def extract_word_from_filename(filename):
    base_name = os.path.splitext(filename)[0]  # Remove the file extension

    # Find the last word (from last letter to first space)
    last_word = base_name.split()[-1]
    
    # Check if the last word is "None" and then extract the first word before the first underscore
    if last_word.lower() == 'none':
        first_word = base_name.split('_')[0]
        return first_word
    else:
        return last_word

# Loop through the files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):  # Check if the file is an Excel file
        file_path = os.path.join(directory, filename)  # Construct full file path
        df = pd.read_excel(file_path)  # Load the file as a DataFrame

        # Extract the appropriate word from the filename
        extracted_word = extract_word_from_filename(filename)

        # Add a new column to the DataFrame
        df['SourceWord'] = extracted_word

        # Print the file name and the first few rows of its DataFrame
        print(f"Processing file: {filename}, Adding SourceWord: {extracted_word}")

        # Concatenate the current file's DataFrame to the aggregated DataFrame
        aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)

# After looping through all files, you can work with the aggregated DataFrame
print("Aggregated DataFrame:")
print(aggregated_df.head())


Processing file: Afghanistan_ AFG.xlsx, Adding SourceWord: AFG
Processing file: Albania_ ALB.xlsx, Adding SourceWord: ALB
Processing file: Algeria_ DZA.xlsx, Adding SourceWord: DZA
Processing file: American Samoa_ ASM.xlsx, Adding SourceWord: ASM
Processing file: Angola_ AGO.xlsx, Adding SourceWord: AGO
Processing file: Anguilla (United Kingdom)_ None.xlsx, Adding SourceWord: Anguilla (United Kingdom)
Processing file: Antigua and Barbuda_ ATG.xlsx, Adding SourceWord: ATG
Processing file: Argentina_ ARG.xlsx, Adding SourceWord: ARG
Processing file: Armenia_ ARM.xlsx, Adding SourceWord: ARM
Processing file: Aruba_ ABW.xlsx, Adding SourceWord: ABW
Processing file: Australia_ AUS.xlsx, Adding SourceWord: AUS
Processing file: Austria_ AUT.xlsx, Adding SourceWord: AUT
Processing file: Azerbaijan_ AZE.xlsx, Adding SourceWord: AZE
Processing file: Bahamas_ BHS.xlsx, Adding SourceWord: BHS
Processing file: Bangladesh_ BGD.xlsx, Adding SourceWord: BGD
Processing file: Barbados_ BRB.xlsx, Adding 

Processing file: Nauru_ NRU.xlsx, Adding SourceWord: NRU
Processing file: Nepal_ NPL.xlsx, Adding SourceWord: NPL
Processing file: Netherlands_ NLD.xlsx, Adding SourceWord: NLD
Processing file: New Caledonia_ NCL.xlsx, Adding SourceWord: NCL
Processing file: New Zealand_ NZL.xlsx, Adding SourceWord: NZL
Processing file: Nicaragua_ NIC.xlsx, Adding SourceWord: NIC
Processing file: Nigeria_ NGA.xlsx, Adding SourceWord: NGA
Processing file: Niger_ NER.xlsx, Adding SourceWord: NER
Processing file: Niue_ NIU.xlsx, Adding SourceWord: NIU
Processing file: North Macedonia_ MKD.xlsx, Adding SourceWord: MKD
Processing file: Norway_ NOR.xlsx, Adding SourceWord: NOR
Processing file: occupied Palestinian territory_ None.xlsx, Adding SourceWord: occupied Palestinian territory
Processing file: Oman_ OMN.xlsx, Adding SourceWord: OMN
Processing file: Pakistan_ PAK.xlsx, Adding SourceWord: PAK
Processing file: Palau_ PLW.xlsx, Adding SourceWord: PLW
Processing file: Panama_ PAN.xlsx, Adding SourceWord: 

In [7]:
# str halde olan sütunu list of dicts'e çeviriyorum
def parse_str_to_list(string):
    try:
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        return None

# Apply the function to the DataFrame column
aggregated_df['sourceObjects'] = aggregated_df['sourceObjects'].apply(parse_str_to_list)

In [8]:
# Kaç tane organizasyondan bahsedildiğini sayıyorum ve yeni kolon olarak ekliyorum
def count_organization_types(obj_list):
    return sum(1 for obj in obj_list if obj.get('type') == 'Organization')

# Apply the function to each row and add the count as a new column
aggregated_df['Organization Count'] = aggregated_df['sourceObjects'].apply(count_organization_types)

In [9]:
# Dictlerden organizasyonların infolarını çekiyorum.
def extract_organization_info_conditional(dicts, count):
    if count != 1:
        return {'Organization': '', 'Organization Type': '', 'Organization Sub Type': '', 'Organization Level': ''}
    for d in dicts:
        if d['type'] == 'Organization':
            return {
                'Organization': d.get('name', ''),
                'Organization Type': ', '.join(d.get('organizationTypes', [])),
                'Organization Sub Type': ', '.join(d.get('organizationSubTypes', [])),
                'Organization Level': ', '.join(d.get('organizationLevels', []))
            }
    return {'Organization': '', 'Organization Type': '', 'Organization Sub Type': '', 'Organization Level': ''}

# Apply the function conditionally based on 'Organization Count'
aggregated_df[['Organization', 'Organization Type', 'Organization Sub Type', 'Organization Level']] = aggregated_df.apply(
    lambda row: pd.Series(extract_organization_info_conditional(row['sourceObjects'], row['Organization Count'])),
    axis=1
)

In [10]:
aggregated_df.head()

,Unnamed: 0,id,amountUSD,budgetYear,childFlowIds,contributionType,createdAt,date,decisionDate,description,...,boundary,onBoundary,reportDetails,refCode,SourceWord,Organization Count,Organization,Organization Type,Organization Sub Type,Organization Level
0,0.0,100131.0,621891.0,2013.0,NaN,financial,2012-11-12T16:26:00Z,2012-07-10T00:00:00Z,2012-07-10T00:00:00Z,Multilateral contribution to UNMAS: Support to...,...,incoming,single,"[{'sourceType': 'Primary', 'organization': 'It...",AID 9908/01/0,AFG,1,"Italy, Government of",Governments,National Governments,National and Sub-National State Actors
1,1.0,100144.0,90928.0,2012.0,NaN,financial,2012-11-13T10:33:00Z,2012-05-01T00:00:00Z,2012-05-01T00:00:00Z,Emergency WASH support to schools in drought i...,...,incoming,single,"[{'sourceType': 'Primary', 'organization': 'Te...",NaN,AFG,1,"United Kingdom, Government of",Governments,National Governments,National and Sub-National State Actors
2,2.0,100145.0,47680.0,2012.0,NaN,financial,2012-11-13T10:37:00Z,2012-05-01T00:00:00Z,2012-05-01T00:00:00Z,Emergency Support to Drought and floods affect...,...,incoming,single,"[{'sourceType': 'Primary', 'organization': 'Te...",NaN,AFG,1,"United Kingdom, Government of",Governments,National Governments,National and Sub-National State Actors
3,3.0,100146.0,1358086.0,2012.0,NaN,financial,2012-11-13T10:43:00Z,2012-09-01T00:00:00Z,2012-09-01T00:00:00Z,Kandahar and Jawzjan Integrated and Sustainabl...,...,incoming,single,"[{'sourceType': 'Primary', 'organization': 'Te...",NaN,AFG,1,"United States of America, Government of",Governments,National Governments,National and Sub-National State Actors
4,4.0,100147.0,117952.0,2012.0,NaN,financial,2012-11-13T10:44:00Z,2012-09-01T00:00:00Z,2012-09-01T00:00:00Z,Kandahar and Jawzjan Integrated and Sustainabl...,...,incoming,single,"[{'sourceType': 'Primary', 'organization': 'Te...",NaN,AFG,1,"United States of America, Government of",Governments,National Governments,National and Sub-National State Actors


In [14]:
aggregated_df[aggregated_df['id']==100131].head()

,Unnamed: 0,id,amountUSD,budgetYear,childFlowIds,contributionType,createdAt,date,decisionDate,description,...,boundary,onBoundary,reportDetails,refCode,SourceWord,Organization Count,Organization,Organization Type,Organization Sub Type,Organization Level
0,0.0,100131.0,621891.0,2013.0,NaN,financial,2012-11-12T16:26:00Z,2012-07-10T00:00:00Z,2012-07-10T00:00:00Z,Multilateral contribution to UNMAS: Support to...,...,incoming,single,"[{'sourceType': 'Primary', 'organization': 'It...",AID 9908/01/0,AFG,1,"Italy, Government of",Governments,National Governments,National and Sub-National State Actors
145905,8.0,100131.0,621891.0,2013.0,NaN,financial,2012-11-12T16:26:00Z,2012-07-10T00:00:00Z,2012-07-10T00:00:00Z,Multilateral contribution to UNMAS: Support to...,...,outgoing,single,"[{'sourceType': 'Primary', 'organization': 'It...",AID 9908/01/0,ITA,1,"Italy, Government of",Governments,National Governments,National and Sub-National State Actors


In [17]:
final_df = aggregated_df[['id','Organization','SourceWord','description','boundary',
                          'Organization Type','Organization Sub Type','Organization Level',
                          'amountUSD','contributionType','method','date','status','Organization Count']]

In [19]:
final_df = final_df[final_df['boundary']=='incoming']
final_df.head()

,id,Organization,SourceWord,description,boundary,Organization Type,Organization Sub Type,Organization Level,amountUSD,contributionType,method,date,status,Organization Count
0,100131.0,"Italy, Government of",AFG,Multilateral contribution to UNMAS: Support to...,incoming,Governments,National Governments,National and Sub-National State Actors,621891.0,financial,Traditional aid,2012-07-10T00:00:00Z,commitment,1
1,100144.0,"United Kingdom, Government of",AFG,Emergency WASH support to schools in drought i...,incoming,Governments,National Governments,National and Sub-National State Actors,90928.0,financial,Traditional aid,2012-05-01T00:00:00Z,paid,1
2,100145.0,"United Kingdom, Government of",AFG,Emergency Support to Drought and floods affect...,incoming,Governments,National Governments,National and Sub-National State Actors,47680.0,financial,Traditional aid,2012-05-01T00:00:00Z,paid,1
3,100146.0,"United States of America, Government of",AFG,Kandahar and Jawzjan Integrated and Sustainabl...,incoming,Governments,National Governments,National and Sub-National State Actors,1358086.0,financial,Traditional aid,2012-09-01T00:00:00Z,paid,1
4,100147.0,"United States of America, Government of",AFG,Kandahar and Jawzjan Integrated and Sustainabl...,incoming,Governments,National Governments,National and Sub-National State Actors,117952.0,financial,Traditional aid,2012-09-01T00:00:00Z,paid,1


In [21]:
final_df = final_df.rename(columns={"id": "flow_id", "SourceWord": "country_iso3"})
final_df.head()

,flow_id,Organization,country_iso3,description,boundary,Organization Type,Organization Sub Type,Organization Level,amountUSD,contributionType,method,date,status,Organization Count
0,100131.0,"Italy, Government of",AFG,Multilateral contribution to UNMAS: Support to...,incoming,Governments,National Governments,National and Sub-National State Actors,621891.0,financial,Traditional aid,2012-07-10T00:00:00Z,commitment,1
1,100144.0,"United Kingdom, Government of",AFG,Emergency WASH support to schools in drought i...,incoming,Governments,National Governments,National and Sub-National State Actors,90928.0,financial,Traditional aid,2012-05-01T00:00:00Z,paid,1
2,100145.0,"United Kingdom, Government of",AFG,Emergency Support to Drought and floods affect...,incoming,Governments,National Governments,National and Sub-National State Actors,47680.0,financial,Traditional aid,2012-05-01T00:00:00Z,paid,1
3,100146.0,"United States of America, Government of",AFG,Kandahar and Jawzjan Integrated and Sustainabl...,incoming,Governments,National Governments,National and Sub-National State Actors,1358086.0,financial,Traditional aid,2012-09-01T00:00:00Z,paid,1
4,100147.0,"United States of America, Government of",AFG,Kandahar and Jawzjan Integrated and Sustainabl...,incoming,Governments,National Governments,National and Sub-National State Actors,117952.0,financial,Traditional aid,2012-09-01T00:00:00Z,paid,1


In [22]:

final_df.tail()

,flow_id,Organization,country_iso3,description,boundary,Organization Type,Organization Sub Type,Organization Level,amountUSD,contributionType,method,date,status,Organization Count
430870,99685.0,"United States of America, Government of",ZWE,"Water, Sanitation and Hygiene (USAID/OFDA)",incoming,Governments,National Governments,National and Sub-National State Actors,341540.0,financial,Traditional aid,2011-06-21T00:00:00Z,commitment,1
430871,99686.0,"United States of America, Government of",ZWE,"Water, Sanitation and Hygiene - through USAID ...",incoming,Governments,National Governments,National and Sub-National State Actors,498116.0,financial,Traditional aid,2011-08-07T00:00:00Z,commitment,1
430872,99687.0,"United States of America, Government of",ZWE,"Natural And Technological Risks, Agriculture a...",incoming,Governments,National Governments,National and Sub-National State Actors,1553381.0,financial,Traditional aid,2012-06-04T00:00:00Z,commitment,1
430873,99793.0,European Commission's Humanitarian Aid and Civ...,ZWE,Emergency WASH Response by Environmental Healt...,incoming,Multilateral Organizations,Other Multilateral Organizations,International Actors,2600000.0,financial,Traditional aid,2011-12-14T00:00:00Z,commitment,1
430874,99938.0,European Commission's Humanitarian Aid and Civ...,ZWE,Environmental Health Alliance (ECHO/ZWE/BUD/20...,incoming,Multilateral Organizations,Other Multilateral Organizations,International Actors,1287001.0,financial,Traditional aid,2011-12-14T00:00:00Z,commitment,1


In [23]:
final_df.to_excel(directory+'FTS_clean_data_latest.xlsx')